In [64]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install tqdm

In [65]:
import bs4 as bs
import requests
from tqdm import tqdm
import pandas as pd

### Step 1: Web scraping table using Beatiful Soup & Requests

In [66]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies') 
resp = resp.text                      
soup = bs.BeautifulSoup(resp, 'lxml')  
table = soup.find('table', {'class': 'wikitable sortable'})  
                                                             

In [131]:
tickers = []
companies = []
industries = []
headquarters = []
dates_1st_added = []

for row in tqdm(table.findAll('tr')[1:]):
    ticker = row.findAll('td')[0].text
    company = row.findAll('td')[2].text
    industry = row.findAll('td')[4].text
    headquarter = row.findAll('td')[4].text
    date_1st_added = row.findAll('td')[5].text

    tickers.append(ticker)
    companies.append(company)
    industries.append(industry)
    headquarters.append(headquarter)
    dates_1st_added.append(date_1st_added)

100%|██████████| 503/503 [00:00<00:00, 23953.85it/s]


In [132]:
df = pd.DataFrame(list(zip(tickers, companies, industries, headquarters, dates_1st_added)), 
                  columns =['Symbol', 'Company', 'Sector', 'Headquarter', 'Year First Added'])
df = df.reset_index(drop=True)
df

,Symbol,Company,Sector,Headquarter,Year First Added
0,MMM\n,Industrials,"Saint Paul, Minnesota","Saint Paul, Minnesota",1957-03-04
1,AOS\n,Industrials,"Milwaukee, Wisconsin","Milwaukee, Wisconsin",2017-07-26
2,ABT\n,Health Care,"North Chicago, Illinois","North Chicago, Illinois",1957-03-04
3,ABBV\n,Health Care,"North Chicago, Illinois","North Chicago, Illinois",2012-12-31
4,ACN\n,Information Technology,"Dublin, Ireland","Dublin, Ireland",2011-07-06
...,...,...,...,...,...
498,XYL\n,Industrials,"White Plains, New York","White Plains, New York",2011-11-01
499,YUM\n,Consumer Discretionary,"Louisville, Kentucky","Louisville, Kentucky",1997-10-06
500,ZBRA\n,Information Technology,"Lincolnshire, Illinois","Lincolnshire, Illinois",2019-12-23
501,ZBH\n,Health Care,"Warsaw, Indiana","Warsaw, Indiana",2001-08-07


### Step 2: Data Cleaning

In [76]:
def clean_columns(x):
    return x.replace("\n", "")

def clean_foundation(x):
    return x.split('(')[-1].split(')')[0][-4:]

def get_state_country(x):
    return x.split(', ')[-1]

In [138]:
df["Symbol"] = df["Symbol"].apply(lambda x: clean_columns(x))
df["Year First Added"] = df["Year First Added"].apply(lambda x: clean_columns(x))

df["Year First Added"] = df["Year First Added"].apply(lambda x: x[:4] if isinstance(x, str) else str(x)[:4])

df["Year First Added"] = df["Year First Added"].replace([''], '0').astype(int)

df["Headquarter"] = df["Headquarter"].apply(lambda x: get_state_country(x))

df = df.drop(columns=['Sector'])

df

,Symbol,Company,Headquarter,Year First Added
0,MMM,Industrials,"Saint Paul, Minnesota",1957
1,AOS,Industrials,"Milwaukee, Wisconsin",2017
2,ABT,Health Care,"North Chicago, Illinois",1957
3,ABBV,Health Care,"North Chicago, Illinois",2012
4,ACN,Information Technology,"Dublin, Ireland",2011
...,...,...,...,...
498,XYL,Industrials,"White Plains, New York",2011
499,YUM,Consumer Discretionary,"Louisville, Kentucky",1997
500,ZBRA,Information Technology,"Lincolnshire, Illinois",2019
501,ZBH,Health Care,"Warsaw, Indiana",2001


### Step 3: Save DataFrame in .csv format to visualize data

In [139]:
df.to_csv('Output/s&p500.csv', index=False)